In [ ]:
import json, nltk
import pandas as pd
from collections import defaultdict
from tqdm import tqdm
import os

In [ ]:
#Need to change these below
INPUT_FILE = '/datasets/wiki-dump/wiki-pages/'
TITLES = '/datasets/wiki-dump/title_index/'
BODY = '/datasets/wiki-dump/body_index/'
# INDEX_STAT = '​invertedindex_stat.txt​'
TOTAL_TOKENS = 0
TOTAL_INV_TOKENS = 0
indexMap = defaultdict(list)
file_num = 0
pages = 1

In [ ]:
train = []
raw = open('paper_dev.jsonl', 'r')
for line in raw:
    obj = json.loads(line)
    train.append(obj)
raw.close()
df = pd.DataFrame(train)
del train
df

,id,verifiable,label,claim,evidence
0,91198,NOT VERIFIABLE,NOT ENOUGH INFO,Colin Kaepernick became a starting quarterback...,"[[[108548, None, None, None]]]"
1,194462,NOT VERIFIABLE,NOT ENOUGH INFO,Tilda Swinton is a vegan.,"[[[227768, None, None, None]]]"
2,137334,VERIFIABLE,SUPPORTS,Fox 2000 Pictures released the film Soul Food.,"[[[289914, 283015, Soul_Food_-LRB-film-RRB-, 0..."
3,166626,NOT VERIFIABLE,NOT ENOUGH INFO,Anne Rice was born in New Jersey.,"[[[191656, None, None, None], [191657, None, N..."
4,111897,VERIFIABLE,REFUTES,Telemundo is a English-language television net...,"[[[131371, 146144, Telemundo, 0]], [[131371, 1..."
...,...,...,...,...,...
9994,113783,NOT VERIFIABLE,NOT ENOUGH INFO,Heavy Metal music was first developed in the U...,"[[[284225, None, None, None], [285168, None, N..."
9995,55560,NOT VERIFIABLE,NOT ENOUGH INFO,Bethany Hamilton was a philosopher.,"[[[71922, None, None, None]]]"
9996,33414,NOT VERIFIABLE,NOT ENOUGH INFO,Mike Huckabee hosts Oprah.,"[[[49508, None, None, None]]]"
9997,208436,NOT VERIFIABLE,NOT ENOUGH INFO,Excuse My French is a studio EP.,"[[[246624, None, None, None]]]"


In [ ]:
train = []
raw = open('/datasets/wiki-dump/wiki-pages/wiki-002.jsonl', 'r')
for line in raw:
    obj = json.loads(line)
    train.append(obj)
raw.close()
df_wiki = pd.DataFrame(train)
del train
df_wiki

,id,text,lines
0,1996_BellSouth_Open,The 1996 BellSouth Open was a tennis tournamen...,0\tThe 1996 BellSouth Open was a tennis tourna...
1,1994_Austrian_motorcycle_Grand_Prix,The 1994 Austrian motorcycle Grand Prix was th...,0\tThe 1994 Austrian motorcycle Grand Prix was...
2,1999_Detroit_Lions_season,The 1999 Detroit Lions season was their 70th i...,0\tThe 1999 Detroit Lions season was their 70t...
3,1985–86_Iraqi_National_League,The 1985 -- 86 Iraqi National League was the 1...,0\tThe 1985 -- 86 Iraqi National League was th...
4,1951_in_Malaya,1951 in Malaya,0\t1951 in Malaya\n1\t
...,...,...,...
49995,1992_Eastern_Michigan_Eagles_football_team,The 1992 Eastern Michigan Eagles football team...,0\tThe 1992 Eastern Michigan Eagles football t...
49996,100_Wayz,100 Wayz is the seventh studio album by rap gr...,0\t100 Wayz is the seventh studio album by rap...
49997,1970_in_film,The year 1970 in film involved some significan...,0\tThe year 1970 in film involved some signifi...
49998,1981_Australian_Drivers'_Championship,The 1981 Australian Drivers ' Championship was...,0\tThe 1981 Australian Drivers ' Championship ...


In [ ]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.structured_prediction
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/elmo-constituency-parser-2020.02.10.tar.gz")

In [ ]:
def extract_proper_nouns(predictor, sent):
    open_flag = 0
    nouns = []
    tagset = ''
    prediction = predictor.predict(
        sentence=sent
    )
    for i in prediction['trees']:
        if i == '(' and not open_flag:
            open_flag = 1
        elif i == '(':
            tagset = ''
        elif i == ')':
            if tagset != '':
                temp = tagset.split(' ')
                tag = temp[0]
                if tag.startswith('NNP'):
                    nouns.append(temp[1])
            open_flag = 0
            tagset = ''
        elif open_flag:
            tagset+=i
    return nouns

In [ ]:
def split_titles(txt):
    return txt.split('_')

In [ ]:
def check_titles(df, nouns,title_score, fstr, title_weight = 10, title_fault = 2):
    
    titles = df.id.apply(split_titles)
    itr = 0
    title_list = []
    for lst in titles:
        lst = list(set(lst))
        for i in range(len(nouns)):
            for j in lst:
                if nouns[i] == j:
                    title_score[fstr+'-'+str(itr)] += title_weight
                else:
                    title_score[fstr+'-'+str(itr)] -= title_fault
        itr += 1
    return title_score

In [ ]:
def check_n_lines(df, n, nouns, title_score, fstr,n_score = 0.1):
    lines = df.lines
    itr = 0
    title_list = []
    for article in lines:
        n_lines = article.split('\n')[:n]
        lst = []
        for line in n_lines:
            lst += line.split()[1:]
        lst = list(lst)
        for i in range(len(nouns)):
            for j in lst:
                if nouns[i] == j:
                    title_score[fstr+'-'+str(itr)] += n_score
        itr += 1
    return title_score

In [ ]:
def check_wiki(code):
    fstr = code.split('-')[0]
    doc = code.split('-')[1]
    train = []
    raw = open('/datasets/wiki-dump/wiki-pages/wiki-'+fstr+'.jsonl', 'r')
    for line in raw:
        obj = json.loads(line)
        train.append(obj)
    raw.close()
    df_wiki = pd.DataFrame(train)
    del train
    itr = 0
    title = df_wiki.id[int(doc)]
    sentences = df_wiki.lines[int(doc)].split('\n')
    return title, sentences

In [ ]:
def get_n_articles(fact, num, flag):
    nouns= extract_proper_nouns(predictor, fact)
    print("Proper nouns",nouns)
    fstr = '001'
    print("Searching database")
    title_score = defaultdict(int)
    temp_score = defaultdict(int)
    offset = open(TITLES+'offset_temp2','r', encoding="utf8", errors='ignore')
    ind = open(TITLES+'index_temp2','r', encoding="utf8", errors='ignore')
    for word in nouns:
        postlist = search(word, offset, ind).strip().split(' ')
        for post in postlist:
            temp = post.split('-')
            title_score[temp[0]+'-'+temp[1]] += 10 - float(temp[3]) * 0.5 * flag
            temp_score[temp[0]+'-'+temp[1]] += float(temp[3])
            # print(temp[2])
    offset = open(BODY+'offset_temp2','r', encoding="utf8", errors='ignore')
    ind = open(BODY+'index_temp2','r', encoding="utf8", errors='ignore')
    for word in nouns:
        postlist = search(word, offset, ind).strip().split(' ')
        # print(postlist)
        for post in postlist:
            temp = post.split('-')
            try:
                title_score[temp[0]+'-'+temp[1]] += int(temp[2]) * 0.1
            # except:
            #     pass
            except:
                print(temp)
    return sorted(title_score.items(), key=lambda item: item[1], reverse= True), temp_score
    sorted_score = sorted(title_score.items(), key=lambda item: item[1], reverse= True)
    res = {}
    for i in sorted_score[0:num]:
        key, value = check_wiki(i[0])
        res[key] = value
    return res

In [ ]:
def search(token,f, ind):
    # Compute filesize of open file sent to us
    global indexsize
    indexsize = os.fstat(f.fileno()).st_size
    hi = indexsize
    lo=0
    x=token
    while lo < hi:
        mid = (lo+hi)//2
        f.seek(mid)
        while f.read(1) != '\n':
            pass
        # print("Newline found")
        line = f.readline()
        row=line.split(':')
        midval = row[0]
        # print(midval)
        if midval < x:
            lo=mid
        elif midval > x: 
            hi=mid
        else:
            offsetval = int(row[1])
            # print(offsetval)
            
            ind.seek(offsetval)
            line = ind.readline()
            post = line.split(':')[1]
            return post
    return False

In [ ]:
for i in get_n_articles('Fox 2000 Pictures released the film Soul Food.', 5, 0)[0]:
    print(check_wiki(i[0])[0], i[1], i[0])

Proper nouns ['Fox', 'Pictures', 'Soul', 'Food']
Searching database
Fox_Searchlight_Pictures 21.200000000000003 036-42054
Soul_Food_-LRB-film-RRB- 21.100000000000005 091-40664
Soul_Food_-LRB-TV_series-RRB- 20.6 090-30971
List_of_Soul_Food_episodes 20.6 060-16273
Soul_Food_-LRB-soundtrack-RRB- 20.4 091-46389
Soul_Food 20.3 091-6086
Rejoice!_Musical_Soul_Food 20.200000000000003 083-12031
Soul_Food_-LRB-Oblivians_album-RRB- 20.200000000000003 090-531
Soul_Food_-LRB-Goodie_Mob_album-RRB- 20.200000000000003 090-5787
Soul_Food_Junkies 20.200000000000003 090-42703
Soul_Food_Café 20.200000000000003 091-8243
Soul_Food_-LRB-Def_Jef_album-RRB- 20.200000000000003 091-28714
Soul_Food_Taqueria 20.200000000000003 091-37890
Soul_Food_-LRB-Bobby_Timmons_album-RRB- 20.200000000000003 091-48861
Fox_Sports_Go 12.9 036-39489
Fox_College_Football 12.1 036-38099
Fox_UFC 12.1 035-42712
William_Fox_-LRB-producer-RRB- 11.9 106-44592
Fox_NFL 11.9 036-18740
Fox_Networks_Group_Benelux 11.9 037-22534
Fox_Networks_G

KeyboardInterrupt: 

In [ ]:
for i in get_n_articles('Fox 2000 Pictures released the film Soul Food.', 5, 0)[0]:
    if i[0] == '091-40664':
        print(i[1])

Proper nouns ['Fox', 'Pictures', 'Soul', 'Food']
Searching database
20.0


In [ ]:
get_n_articles('Fox 2000 Pictures released the film Soul Food.', 5, 1)[1]['091-40664']

Proper nouns ['Fox', 'Pictures', 'Soul', 'Food']
Searching database


41.0

In [ ]:
df_dict = df.to_dict(orient='list')
flag = 0
count = 0
for i in range(len(df_dict['id'])):
    if df_dict['verifiable'][i] == 'VERIFIABLE':
        print(df_dict['id'][i], df_dict['claim'][i])
        claim = df_dict['claim'][i]
        temp = df_dict['evidence'][i]
        evidence_titles = []
        for i in temp:
            for j in i:
                evidence_titles.append(j[2])
        evidence_titles = list(set(evidence_titles))
        # print(evidence_titles)
        print()
        flag += 1
        articles = get_n_articles(claim, 10)
        article_titles = list(articles.keys())
        print(article_titles, evidence_titles)
        if len(list(set(article_titles) & set(evidence_titles))) > 0:
            count+= 1
            print("Found")
        if flag == 50:
            break

print(flag, count)

137334 Fox 2000 Pictures released the film Soul Food.

Proper nouns ['Fox', 'Pictures', 'Soul', 'Food']
Searching database


AttributeError: 'list' object has no attribute 'keys'

In [ ]:
check_wiki('090-30971')

('Soul_Food_-LRB-TV_series-RRB-',
 ['0\tSoul Food is a television drama that aired Wednesday nights on Showtime from June 28 , 2000 to May 26 , 2004 .\tShowtime\tShowtime (TV network)',
  "1\tCreated by George Tillman , Jr. and developed for television by Felicia D. Henderson , Soul Food is based upon Tillman 's childhood experiences growing up in Wisconsin , and is a continuation of his 1997 film of the same name .\tFelicia D. Henderson\tFelicia D. Henderson\tWisconsin\tWisconsin\tfilm of the same name\tSoul Food (film)",
  '2\tHaving aired for 74 episodes , it is the longest running drama with a predominantly black cast in the history of North American prime-time television .\tblack\tblack people',
  '3\t'])

In [ ]:
df[df.verifiable == 'VERIFIABLE'].claim[22]